# SVM_reject

In [17]:
import scanpy as sc
import os 
import numpy as np
import pandas as pd
import time as tm
from sklearn.svm import LinearSVC
import scanpy as sc
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
from sklearn.calibration import CalibratedClassifierCV

In [7]:
adata = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/scgpt_dataset/demo_train.h5ad")

In [8]:
adata

AnnData object with n_obs × n_vars = 10600 × 3000
    obs: 'Celltype', 'celltype'
    var: 'Gene Symbol'

In [9]:
eplison = 10
np.random.seed(2023)
rand_data = np.asarray(np.random.rand(adata.X.shape[0], adata.X.shape[1]))

In [10]:
X_copy = adata.X.toarray()

In [12]:
X_posioned = X_copy + eplison * rand_data

In [13]:
adata.layers['poi'] = X_posioned 

In [19]:
X_pos_train, X_pos_test, y_pos_train, y_pos_test =sklearn.model_selection.train_test_split(X_posioned, adata.obs['celltype'] , test_size=0.33, random_state=2023)

In [21]:
X_train = np.vstack([X_copy, X_pos_train])
y_train = np.hstack([adata.obs['celltype'].values, y_pos_train])

In [24]:
Classifier = LinearSVC()
clf = CalibratedClassifierCV(Classifier)
clf.fit(X_train, y_train)

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust_gcn/lib/python3.8/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/gpfs/gibbs/projec

CalibratedClassifierCV(estimator=LinearSVC())

In [25]:
predicted_y = clf.predict(X_pos_test)
print(classification_report(y_pos_test, predicted_y, digits=4))

              precision    recall  f1-score   support

          PP     0.5287    0.3866    0.4466       119
         PSC     0.7917    0.6746    0.7284       169
      acinar     0.6767    0.6534    0.6649       378
       alpha     0.7418    0.8292    0.7830      1036
        beta     0.6871    0.7022    0.6946      1004
       delta     0.5183    0.4363    0.4738       259
      ductal     0.6079    0.5976    0.6027       410
 endothelial     0.9250    0.8409    0.8810        88
     epsilon     1.0000    0.1429    0.2500         7
  macrophage     0.8889    0.5714    0.6957        14
        mast     0.6667    0.4000    0.5000         5
     schwann     0.6667    0.4000    0.5000         5
      t_cell     1.0000    0.2500    0.4000         4

    accuracy                         0.6910      3498
   macro avg     0.7461    0.5296    0.5862      3498
weighted avg     0.6878    0.6910    0.6866      3498



# Neural Network

In [1]:
import scanpy as sc
import os 
import numpy as np
import pandas as pd
import time as tm
from sklearn.svm import LinearSVC
import scanpy as sc
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
from sklearn.calibration import CalibratedClassifierCV

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
adata = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/scgpt_dataset/demo_train.h5ad")

In [3]:
adata

AnnData object with n_obs × n_vars = 10600 × 3000
    obs: 'Celltype', 'celltype'
    var: 'Gene Symbol'

In [4]:
eplison = 10
np.random.seed(2023)
rand_data = np.asarray(np.random.rand(adata.X.shape[0], adata.X.shape[1]))

In [5]:
X_copy = adata.X.toarray()

In [6]:
X_posioned = X_copy + eplison * rand_data

In [7]:
adata.layers['poi'] = X_posioned 

In [8]:
X_pos_train, X_pos_test, y_pos_train, y_pos_test =sklearn.model_selection.train_test_split(X_posioned, adata.obs['celltype'] , test_size=0.33, random_state=2023)

In [9]:
X_train = np.vstack([X_copy, X_pos_train])
y_train = np.hstack([adata.obs['celltype'].values, y_pos_train])

In [10]:
import torch
import torch.nn.functional as F
import lightning as L
import os
import pandas as pd
import numpy as np
import pickle
import sklearn.model_selection
import scipy.stats
import sklearn.metrics
import scanpy as sc

from torch.utils.data import DataLoader
from torch import nn
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping


In [11]:
layers =[adata.X.shape[1], 512, len(set(adata.obs['Celltype']))]

In [12]:
# layers

In [13]:
class Encoder(nn.Module):
    def __init__(self, k=3):
        super().__init__()
        self.shared_layers = nn.Sequential(nn.Linear(layers[0], layers[1]), 
                                nn.BatchNorm1d(layers[1]),
                                nn.ReLU(), 
                                nn.Dropout(input_dropout),
                                nn.Linear(layers[1], layers[1]),
                                nn.BatchNorm1d(layers[1]),
                                nn.ReLU(), 
                                nn.Dropout(input_dropout),
                                nn.Linear(layers[1], layers[2])
                               )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        
        return self.softmax (self.shared_layers(x))
        



class LitAutoEncoder(L.LightningModule):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder
        self.loss = nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        
        output_label = self.encoder(x)
        
        loss = self.loss(output_label, y.long())
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch
        
        output_label = self.encoder(x)
        
        val_loss = self.loss(output_label, y.long())
        
        self.log("val_loss", val_loss.item())
        return val_loss

        
    def test_step(self, batch, batch_idx):
        # this is the test loop
        x, y = batch
        
        output_label = self.encoder(x)
        
        test_loss = self.loss(output_label, y.long())
        self.log("test_loss", test_loss.item())
        return test_loss
        
    def forward(self, x):
        return self.encoder(x)

    def configure_optimizers(self):
            optimizer = torch.optim.Adam(
                params=self.parameters(), 
                lr=eta
)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer,
                patience=10,
                verbose=True
            )
            return {
               'optimizer': optimizer,
               'lr_scheduler': scheduler, # Changed scheduler to lr_scheduler
               'monitor': 'val_loss'
           }


In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [15]:
label_data = le.fit_transform(y_train)

In [16]:
adata.X

<10600x3000 sparse matrix of type '<class 'numpy.float32'>'
	with 3227742 stored elements in Compressed Sparse Row format>

In [17]:
import sklearn.model_selection 

In [18]:
X_tr, X_val, y_tr, y_val = sklearn.model_selection.train_test_split(X_train,label_data , random_state=2023)

In [19]:
X_test = X_pos_test
y_test = le.transform(y_pos_test)

In [21]:
X_tr, X_val, X_train, X_test, y_tr, y_val, y_train, y_test =torch.FloatTensor(X_tr),torch.FloatTensor(X_val),torch.FloatTensor(X_tr),torch.FloatTensor(X_test),torch.FloatTensor(y_tr), torch.FloatTensor(y_val), torch.FloatTensor(y_tr), torch.FloatTensor(y_test)


train_dataset = torch.utils.data.TensorDataset(X_tr, y_tr)
valid_dataset = torch.utils.data.TensorDataset(X_val, y_val)
test_dataset = torch.utils.data.TensorDataset(X_tr, y_tr)

epochs = 1000 
act_func = 'GELU'
dropout = 0.5 
input_dropout = 0.2
eta = 1e-4
norm = 'tanh' 
drug_num_dim = 16

model = LitAutoEncoder(Encoder())

print(model.log) #see log and log dir for model place

lr_monitor = LearningRateMonitor(logging_interval='step')

train_loader = DataLoader(train_dataset, batch_size=4096, num_workers=5, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=2048, num_workers=5)

# train with both splits
trainer = L.Trainer(callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=100)], max_epochs=1000)

trainer.log_dir

trainer.fit(model, train_loader, valid_loader)

trainer.test(model, dataloaders=DataLoader(test_dataset, batch_size=1024, num_workers=5))

# checkpoint = torch.load("/gpfs/gibbs/pi/zhao/tl688/adversial_training/lightning_logs/version_24415172/checkpoints/epoch=399-step=800.ckpt")
# model.load_state_dict(checkpoint["state_dict"])


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


<bound method LightningModule.log of LitAutoEncoder(
  (encoder): Encoder(
    (shared_layers): Sequential(
      (0): Linear(in_features=3000, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=512, out_features=512, bias=True)
      (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.2, inplace=False)
      (8): Linear(in_features=512, out_features=13, bias=True)
    )
    (softmax): Softmax(dim=1)
  )
  (loss): CrossEntropyLoss()
)>


/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.8 /gpfs/gibbs/project/zhao/tl688/conda_envs/deeprob ...
You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    1.7088230848312378     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.7088230848312378}]

In [22]:
for m in model.encoder.modules():

    for child in m.children():

        if type(child) == nn.BatchNorm1d:

            child.track_running_stats = False

            child.running_mean = None

            child.running_var = None
model.encoder.eval()

Encoder(
  (shared_layers): Sequential(
    (0): Linear(in_features=3000, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=512, out_features=13, bias=True)
  )
  (softmax): Softmax(dim=1)
)

In [23]:
X_test = torch.FloatTensor(X_pos_test)
y_test = torch.FloatTensor(le.transform(y_pos_test))

In [24]:
with torch.no_grad():
    y_pred = model.encoder(X_test)
    _ , y_pred = torch.max(y_pred, 1)

In [25]:
y_pred

tensor([4, 3, 4,  ..., 6, 3, 3])

In [26]:
y_test

tensor([4., 3., 4.,  ..., 6., 3., 3.])

In [27]:
sklearn.metrics.classification_report(y_test, y_pred)

/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/gpfs/gibbs/project/zhao/tl688/conda_envs/deeprobust/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

'              precision    recall  f1-score   support\n\n         0.0       0.67      0.54      0.60       119\n         1.0       0.89      0.93      0.91       169\n         2.0       0.90      0.91      0.91       378\n         3.0       0.90      0.93      0.92      1036\n         4.0       0.89      0.89      0.89      1004\n         5.0       0.81      0.69      0.74       259\n         6.0       0.82      0.92      0.87       410\n         7.0       0.90      0.82      0.86        88\n         8.0       0.00      0.00      0.00         7\n         9.0       0.00      0.00      0.00        14\n        10.0       0.00      0.00      0.00         5\n        11.0       0.00      0.00      0.00         5\n        12.0       0.00      0.00      0.00         4\n\n    accuracy                           0.87      3498\n   macro avg       0.52      0.51      0.51      3498\nweighted avg       0.86      0.87      0.87      3498\n'